## POS Tagging - Lexicon and Rule Based Taggers

Let's look at the two most basic tagging techniques - lexicon based (or unigram) and rule-based. 

In this guided exercise, you will explore the WSJ (wall street journal) POS-tagged corpus that comes with NLTK and build a lexicon and rule-based tagger using this corpus as the tarining data. 

This exercise is divided into the following sections:
1. Reading and understanding the tagged dataset
2. Exploratory analysis

### 1. Reading and understanding the tagged dataset

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Importing libraries
import nltk
import numpy as np
import pandas as pd
import pprint, time
import random
from sklearn.model_selection import train_test_split
from nltk.tokenize import word_tokenize
import math

In [3]:
nltk.corpus.treebank.tagged_sents()

[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], ...]

In [4]:
# reading the Treebank tagged sentences
wsj = list(nltk.corpus.treebank.tagged_sents())

In [5]:
# samples: Each sentence is a list of (word, pos) tuples
wsj[:3]

[[('Pierre', 'NNP'),
  ('Vinken', 'NNP'),
  (',', ','),
  ('61', 'CD'),
  ('years', 'NNS'),
  ('old', 'JJ'),
  (',', ','),
  ('will', 'MD'),
  ('join', 'VB'),
  ('the', 'DT'),
  ('board', 'NN'),
  ('as', 'IN'),
  ('a', 'DT'),
  ('nonexecutive', 'JJ'),
  ('director', 'NN'),
  ('Nov.', 'NNP'),
  ('29', 'CD'),
  ('.', '.')],
 [('Mr.', 'NNP'),
  ('Vinken', 'NNP'),
  ('is', 'VBZ'),
  ('chairman', 'NN'),
  ('of', 'IN'),
  ('Elsevier', 'NNP'),
  ('N.V.', 'NNP'),
  (',', ','),
  ('the', 'DT'),
  ('Dutch', 'NNP'),
  ('publishing', 'VBG'),
  ('group', 'NN'),
  ('.', '.')],
 [('Rudolph', 'NNP'),
  ('Agnew', 'NNP'),
  (',', ','),
  ('55', 'CD'),
  ('years', 'NNS'),
  ('old', 'JJ'),
  ('and', 'CC'),
  ('former', 'JJ'),
  ('chairman', 'NN'),
  ('of', 'IN'),
  ('Consolidated', 'NNP'),
  ('Gold', 'NNP'),
  ('Fields', 'NNP'),
  ('PLC', 'NNP'),
  (',', ','),
  ('was', 'VBD'),
  ('named', 'VBN'),
  ('*-1', '-NONE-'),
  ('a', 'DT'),
  ('nonexecutive', 'JJ'),
  ('director', 'NN'),
  ('of', 'IN'),
  ('this'

In the list mentioned above, each element of the list is a sentence. Also, note that each sentence ends with a full stop '.' whose POS tag is also a '.'. Thus, the POS tag '.' demarcates the end of a sentence.

Also, we do not need the corpus to be segmented into sentences, but can rather use a list of (word, tag) tuples. Let's convert the list into a (word, tag) tuple.

In [6]:
# converting the list of sents to a list of (word, pos tag) tuples
tagged_words = [tup for sent in wsj for tup in sent]
print(len(tagged_words))
tagged_words[:10]

100676


[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT')]

We now have a list of about 100676 (word, tag) tuples. Let's now do some exploratory analyses.

### 2. Exploratory Analysis

Let's now conduct some basic exploratory analysis to understand the tagged corpus. To start with, let's ask some simple questions:
1. How many unique tags are there in the corpus? 
2. Which is the most frequent tag in the corpus?
3. Which tag is most commonly assigned to the following words:
    - "bank"
    - "executive"


In [7]:
len(set([word[0] for word in tagged_words]))

12408

In [8]:
tagged_words

[('Pierre', 'NNP'),
 ('Vinken', 'NNP'),
 (',', ','),
 ('61', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 (',', ','),
 ('will', 'MD'),
 ('join', 'VB'),
 ('the', 'DT'),
 ('board', 'NN'),
 ('as', 'IN'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('Nov.', 'NNP'),
 ('29', 'CD'),
 ('.', '.'),
 ('Mr.', 'NNP'),
 ('Vinken', 'NNP'),
 ('is', 'VBZ'),
 ('chairman', 'NN'),
 ('of', 'IN'),
 ('Elsevier', 'NNP'),
 ('N.V.', 'NNP'),
 (',', ','),
 ('the', 'DT'),
 ('Dutch', 'NNP'),
 ('publishing', 'VBG'),
 ('group', 'NN'),
 ('.', '.'),
 ('Rudolph', 'NNP'),
 ('Agnew', 'NNP'),
 (',', ','),
 ('55', 'CD'),
 ('years', 'NNS'),
 ('old', 'JJ'),
 ('and', 'CC'),
 ('former', 'JJ'),
 ('chairman', 'NN'),
 ('of', 'IN'),
 ('Consolidated', 'NNP'),
 ('Gold', 'NNP'),
 ('Fields', 'NNP'),
 ('PLC', 'NNP'),
 (',', ','),
 ('was', 'VBD'),
 ('named', 'VBN'),
 ('*-1', '-NONE-'),
 ('a', 'DT'),
 ('nonexecutive', 'JJ'),
 ('director', 'NN'),
 ('of', 'IN'),
 ('this', 'DT'),
 ('British', 'JJ'),
 ('industrial', 'JJ'),
 ('c

In [9]:
# question 1: Find the number of unique POS tags in the corpus
# you can use the set() function on the list of tags to get a unique set of tags, 
# and compute its length
unique_set = set([word[1] for word in tagged_words])
len(unique_set)

46

In [10]:
Counter([word[1] for word in tagged_words])

NameError: name 'Counter' is not defined

In [ ]:
# question 2: Which is the most frequent tag in the corpus
# to count the frequency of elements in a list, the Counter() class from collections
# module is very useful, as shown below

from collections import Counter
tag_counts = Counter([word[1] for word in tagged_words])
tag_counts.most_common(1)

In [ ]:
# the most common tags can be seen using the most_common() method of Counter
tag_counts.most_common(5)

In [ ]:
tag_counts

Thus, NN is the most common tag followed by IN, NNP, DT, -NONE- etc. You can read the exhaustive list of tags using the NLTK documentation as shown below.

In [ ]:
# list of POS tags in NLTK
nltk.help.upenn_tagset()

In [ ]:
# question 3: Which tag is most commonly assigned to the word w. Get the tags list that appear for word w and then use the Counter()
# Try 
w ='bank' 
bank = Counter([word[1] for word in tagged_words if word[0] == w])
bank

In [ ]:
# question 3: Which tag is most commonly assigned to the word w. Try 'executive' 
executive = Counter([word[1] for word in tagged_words if word[0] == 'executive'])
executive

### 2. Exploratory Analysis Contd.

Until now, we were looking at the frequency of tags assigned to particular words, which is the basic idea used by lexicon or unigram taggers. Let's now try observing some rules which can potentially be used for POS tagging. 

To start with, let's see if the following questions reveal something useful:

4. What fraction of words with the tag 'VBD' (verb, past tense) end with the letters 'ed'
5. What fraction of words with the tag 'VBG' (verb, present participle/gerund) end with the letters 'ing'

In [ ]:
# 4. how many words with the tag 'VBD' (verb, past tense) end with 'ed'
# first get the all the words tagged as VBD
past_tense_verbs = [word for word in tagged_words if word[1] =='VBD']

# subset the past tense verbs with words ending with 'ed'. (Try w.endswith('ed'))
ed_verbs = [word for word in past_tense_verbs if word[0].endswith('ed') ]
print(len(ed_verbs) / len(past_tense_verbs))
ed_verbs[:20]

In [ ]:
# 5. how many words with the tag 'VBG' end with 'ing'
participle_verbs = [ word for word in tagged_words if word[1] =='VBG']
ing_verbs = [w for w in participle_verbs if w[0].endswith('ing')]
print(len(ing_verbs) / len(participle_verbs))
ing_verbs[:20]

## 2. Exploratory Analysis Continued

Let's now try observing some tag patterns using the fact the some tags are more likely to apper after certain other tags. For e.g. most nouns NN are usually followed by determiners DT ("The/DT constitution/NN"), adjectives JJ usually precede a noun NN (" A large/JJ building/NN"), etc. 

Try answering the following questions:
1. What fraction of adjectives JJ are followed by a noun NN? 
2. What fraction of determiners DT are followed by a noun NN?
3. What fraction of modals MD are followed by a verb VB?

In [ ]:
# question: what fraction of adjectives JJ are followed by a noun NN

# create a list of all tags (without the words)
tags = [ word[1] for word in tagged_words]

# create a list of JJ tags
jj_tags = [ word[1] for word in tagged_words if word[1] == 'JJ']

# create a list of (JJ, NN) tags
jj_nn_tags = [(tags[index], tags[index+1]) for index, t in enumerate(tags) 
              if tags[index] == 'JJ' and tags[index+1] == 'NN']

print(len(jj_tags))
print(len(jj_nn_tags))
print(len(jj_nn_tags) / len(jj_tags))

In [ ]:
# question: what fraction of determiners DT are followed by a noun NN
dt_tags = [word[1] for word in tagged_words if word[1] == 'DT']
dt_nn_tags = [(tags[index], tags[index+1]) for index, t in enumerate(tags) 
              if tags[index] == 'DT' and tags[index+1] == 'NN']

print(len(dt_tags))
print(len(dt_nn_tags))
print(len(dt_nn_tags) / len(dt_tags))

In [ ]:
# question: what fraction of modals MD are followed by a verb VB?
md_tags = [word[1] for word in tagged_words if word[1] == 'MD']
md_vb_tags = [(tags[index], tags[index+1]) for index, t in enumerate(tags) 
              if tags[index] == 'MD' and tags[index+1] == 'VB']

print(len(md_tags))
print(len(md_vb_tags))
print(len(md_vb_tags) / len(md_tags))

## Lexican and Ruley based models for POS tagging:

#### training and test set:

In [ ]:
random.seed(1234)
train_set, test_set = train_test_split(wsj, test_size=0.3)

In [ ]:
unigram_tagger = nltk.UnigramTagger(train_set)
unigram_tagger.evaluate(test_set)

In [ ]:
bigram_tagger = nltk.BigramTagger(wsj)
bigram_tagger.evaluate(test_set)

#### Rule based tagging: 

In [ ]:
pattern = [
    (r'.*ing$', 'VBG'),
    (r'.*ed$', 'VBD'),
    (r'.*es$', 'VBZ'),
    (r'.*ould$', 'MD'),
    (r'.*\'s$', 'NN$'),
    (r'.*s$', 'NNS'),
    (r'^-?[0-9]+(.[0-9]+)?$', 'CD'),
    (r'.*', 'NN')
]
regexp_tagger = nltk.RegexpTagger(pattern)
regexp_tagger.evaluate(test_set)

#### Combine lexical and rule based models:

In [ ]:
# ====> Combine regexp tagger and unigram tagger
rule_based_taggers = nltk.RegexpTagger(pattern)
lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_taggers)
lexicon_tagger.evaluate(test_set)

#### regexep, unigram and bigram taggers:

In [ ]:
rule_based_taggers = nltk.RegexpTagger(pattern)

unigram_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_taggers)
bigram_tagger = nltk.BigramTagger(train_set, backoff=unigram_tagger)
bigram_tagger.evaluate(test_set)

#### regexp, unigram, bigram and trigram

In [ ]:
class PosTagger:
    
    def __init__(self, train_set, test_set, pattern):
        self.train_set = train_set
        self.test_set = test_set
        self.pattern = pattern
    
    def get_pos_tagger(self):
        rule_based_tagger = nltk.RegexpTagger(pattern)
        unigram_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger)
        bigram_tagger = nltk.BigramTagger(train_set, backoff=unigram_tagger)
        trigram_tagger = nltk.TrigramTagger(train_set, backoff=bigram_tagger)
        self.main_tagger = trigram_tagger
        return unigram_tagger
    
    def evaluate(self):
        return self.main_tagger.evaluate(test_set)

In [ ]:
pos_tagger = PosTagger(train_set, test_set, pattern)
pos_tagger.get_pos_tagger()
pos_tagger.evaluate()

In [ ]:
# ====> Unigram tagger default variation
tagger = nltk.NgramTagger(1, train_set, backoff=regexp_tagger)
tagger.evaluate(test_set)

In [ ]:
wsj = list(nltk.corpus.treebank.tagged_sents())

In [ ]:
wsj[:2]

### POS tagging algorithm - HIDDEN MARKOV MODEL 

In [ ]:
unique_set # ===> tag set

In [ ]:
train_tagged_words = [tu for sentence in train_set for tu in sentence]
len(train_tagged_words)

In [ ]:
tokens = [pair[0] for pair in train_tagged_words]
vocabulary = set(tokens)
len(vocabulary)

### Model

In [ ]:
class HiddenMarkovModel():
    def __init__(self, train_tagged_words, tags):
        self.train_tagged_words = train_tagged_words
        self.tags = tags
    
    def word_given_tag(self, word, tag):
        tags_list = [pair for pair in self.train_tagged_words if pair[1] == tag]
        matching_words = [pair for pair in tags_list if pair[0] == word]
        return (len(matching_words), len(tags_list))

    def transition(self, t2, t1):
        t2_followed_by_t1_count, t1_count = 0, 0
        tags = [word[1] for word in self.train_tagged_words]
        for _idx in range(len(tags) - 1):
            if (tags[_idx] == t1 and tags[_idx+1]  == t2):
                t2_followed_by_t1_count += 1
                t1_count += 1
            elif (tags[_idx] == t1):
                t1_count += 1
        return (t2_followed_by_t1_count, t1_count)

    def construct_transition_matrix(self):
        no_of_tags = len(self.tags)
        tags_matrix = np.zeros((no_of_tags, no_of_tags), dtype='float32')
        for i, t1 in enumerate(list(self.tags)):
            for j, t2 in enumerate(list(self.tags)):
                result = self.transition(t2, t1)
                tags_matrix[i][j] = (result[0]/result[1])
        
        return tags_matrix
    
    def vetarbi(self, words, tags_transistion_matrix):
        result = []
        tokens = list(set([word[1] for word in self.train_tagged_words]))
        
        for i, word in enumerate(words):
            probabilities = []
            for tag in tokens:
                if i == 0:
                    transition_p = tags_transistion.loc['.', tag]
                else:
                    transition_p = tags_transistion.loc[result[-1], tag]

                words_count = self.word_given_tag(word, tag)
                emission_probability = words_count[0] / words_count[1]
                probabilities.append( emission_probability *  transition_p)
            pmax = max(probabilities)
            state_max = tokens[probabilities.index(pmax)]
            result.append(state_max)
        return list(zip(words, result))

In [ ]:
obj = HiddenMarkovModel(train_tagged_words, unique_set)

print("\n large")
print(obj.word_given_tag("large", "JJ"))
print(obj.word_given_tag("large", "VB"))
print(obj.word_given_tag("large", "NN"))

print("\n book")
print(obj.word_given_tag('book', 'NN'))
print(obj.word_given_tag('book', 'JJ'))
print(obj.word_given_tag('book', 'VB'))

In [ ]:
print(obj.transition('NNP', 'JJ'))
print(obj.transition('NN', 'JJ'))
print(obj.transition('VBG', 'NNP'))

In [ ]:
obj.construct_transition_matrix()

In [ ]:
obj.word_given_tag('Android', 'NN')

In [ ]:
result = obj.transition('VB', 'MD')
result[0] /result[1]

In [ ]:
tags_transistion = pd.DataFrame(obj.construct_transition_matrix(), index=list(unique_set), columns=list(unique_set))
tags_transistion

In [ ]:
# ===> start of the the sentence trancistion probabilities
tags_transistion.loc['.', :]

In [ ]:
plt.figure(figsize=(20, 20))
sns.heatmap(tags_transistion)
plt.show()

### Test run of hidden markov model 

In [ ]:
random.seed(1234)

# choose random 5 sents
rndom = [random.randint(1,len(test_set)) for x in range(5)]

# list of sents
test_run = [test_set[i] for i in rndom]

# list of tagged words
test_run_base = [tup for sent in test_run for tup in sent]

# list of untagged words
test_tagged_words = [tup[0] for sent in test_run for tup in sent]
test_run

In [ ]:
start = time.time()
obj = HiddenMarkovModel(train_tagged_words, unique_set)
tagged_seq = obj.vetarbi(test_tagged_words, tags_transistion)
end = time.time()
difference = end-start

In [ ]:
print("Time taken in seconds: ", difference)
print(tagged_seq)

In [ ]:
check = [i for i, j in zip(tagged_seq, test_run_base) if i == j] 
accuracy = len(check)/len(tagged_seq)
accuracy

In [ ]:
incorrect_tagged_cases = [[test_run_base[i-1],j] for i, j in enumerate(zip(tagged_seq, test_run_base)) if j[0]!=j[1]]
incorrect_tagged_cases

In [ ]:
sentence_test = 'Twitter is the best networking social site. Man is a social animal. Data science is an emerging field. Data science jobs are high in demand.'
words = word_tokenize(sentence_test)

start = time.time()
tagged_seq = obj.vetarbi(words, tags_transistion)
end = time.time()
difference = end-start
print(difference)
print(tagged_seq)
print(difference)